In [1]:
import numpy as np
from itertools import combinations, chain
import xcover

In [2]:
def convert_state_to_set(state):
    return set(np.nonzero(state.flatten())[0])

def get_minimal_state(state):
    axii = state.ndim
    for axis in range(axii):
        while 1 not in np.take(state, indices=[0], axis=axis):
            state = np.roll(state, -1, axis=axis)
    return state

def calc_all_rotations(state):
    """
    Returns all unique 90-degree rotations of an n-dimensional numpy array.
    
    Args:
        array: n-dimensional numpy array
        
    Returns:
        List of rotated arrays
    """
    n = state.ndim
    rotations = []
    
    # Get all possible axis pairs for rotation
    axis_pairs = list(combinations(range(n), 2))
    
    def rotate_recursive(curr, depth=0):
        if depth == len(axis_pairs):
            if not any(np.array_equal(curr, r) for r in rotations):
                rotations.append(curr.copy())
            return
            
        # Try all 4 rotations for current axis pair
        temp = curr.copy()
        for _ in range(4):
            rotate_recursive(temp, depth + 1)
            temp = np.rot90(temp, axes=axis_pairs[depth])
    
    rotate_recursive(state)
    return rotations

def calc_all_rolls(state, axis):
    states = []
    while 1 not in np.take(state, indices=[state.shape[axis] - 1], axis=axis):
        state = np.roll(state, 1, axis=axis)
        states.append(state)
    return states

def calc_all_states(state):
    """With rolls and rotations"""
    axii = state.ndim
    states = []
    rotations = calc_all_rotations(state)
    for rot in rotations:
        rot = get_minimal_state(rot)
        rolls = []
        rolls.append(rot)
        for axis in range(axii):
            axis_rolls = []
            for roll in rolls:
                axis_rolls += calc_all_rolls(roll, axis)
            rolls += axis_rolls
        states += rolls
    return states


In [5]:
state = np.array([[1,1,1],[0,0,0],[0,0,0]])
states = calc_all_states(state)

print("States:")
for state_ in states:
    print(state_)
    print("")
    
print("State sets:")
state_sets = [convert_state_to_set(state) for state in states]
print(state_sets)

print("solutions:")
solutions = xcover.covers(state_sets)
print(list(solutions))

States:
[[1 1 1]
 [0 0 0]
 [0 0 0]]

[[0 0 0]
 [1 1 1]
 [0 0 0]]

[[0 0 0]
 [0 0 0]
 [1 1 1]]

[[1 0 0]
 [1 0 0]
 [1 0 0]]

[[0 1 0]
 [0 1 0]
 [0 1 0]]

[[0 0 1]
 [0 0 1]
 [0 0 1]]

[[1 1 1]
 [0 0 0]
 [0 0 0]]

[[0 0 0]
 [1 1 1]
 [0 0 0]]

[[0 0 0]
 [0 0 0]
 [1 1 1]]

[[1 0 0]
 [1 0 0]
 [1 0 0]]

[[0 1 0]
 [0 1 0]
 [0 1 0]]

[[0 0 1]
 [0 0 1]
 [0 0 1]]

State sets:
[{0, 1, 2}, {3, 4, 5}, {8, 6, 7}, {0, 3, 6}, {1, 4, 7}, {8, 2, 5}, {0, 1, 2}, {3, 4, 5}, {8, 6, 7}, {0, 3, 6}, {1, 4, 7}, {8, 2, 5}]
solutions:
[[9, 5, 4], [9, 5, 10], [9, 11, 10], [9, 11, 4], [6, 8, 7], [6, 8, 1], [6, 2, 1], [6, 2, 7], [3, 11, 10], [3, 11, 4], [3, 5, 4], [3, 5, 10], [0, 2, 1], [0, 2, 7], [0, 8, 7], [0, 8, 1]]


In [6]:
state = np.array([[[1,1,1,1,0],[0,1,0,0,0],[0,0,0,0,0]],[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]],[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]])
states = calc_all_states(state)

print("States:")
for state_ in states:
    print(state_)
    print("")
    
print("State sets:")
state_sets = [convert_state_to_set(state) for state in states]
print(state_sets)

print("solutions:")
solutions = xcover.covers([list(s) for s in state_sets])
print(list(solutions))

States:
[[[1 1 1 1 0]
  [0 1 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]

[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 1 1 1 0]
  [0 1 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]

[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[1 1 1 1 0]
  [0 1 0 0 0]
  [0 0 0 0 0]]]

[[[0 0 0 0 0]
  [1 1 1 1 0]
  [0 1 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]

[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [1 1 1 1 0]
  [0 1 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]

[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [1 1 1 1 0]
  [0 1 0 0 0]]]

[[[0 1 1 1 1]
  [0 0 1 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]]

[[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]]

 [[0 1 1 1 1]
  [

In [ ]:
# Check solutions

